In [1]:
import tensorflow as tf
import tf2onnx
from tensorflow.python.keras.saving import saving_utils


In [2]:
def create_AND_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(
        units=2,
        activation='linear',
        input_shape=(2,),
        kernel_initializer=tf.keras.initializers.constant([-7.5]),
        bias_initializer=tf.keras.initializers.constant([5.0, 5.0])
    ))
    # model.save("networks/and-gate")
    print(model)
    return model


In [3]:
def create_OR_model():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(
        units=2,
        activation='linear',
        input_shape=(2,),
        kernel_initializer=tf.keras.initializers.constant([-2.5]),
        bias_initializer=tf.keras.initializers.constant([5.0, 5.0])
    ))
    model.save("networks/or-gate")

In [4]:
def tf_to_onnx(model):
    function = saving_utils.trace_model_call(model)
    concrete_func = function.get_concrete_function()
    input_names = [input_tensor.name for input_tensor in concrete_func.inputs
                   if input_tensor.dtype != tf.dtypes.resource]
    output_names = [output_tensor.name for output_tensor in concrete_func.outputs
                    if output_tensor.dtype != tf.dtypes.resource]
    frozen_graph = tf2onnx.tf_loader.from_function(concrete_func, input_names, output_names)
    with tf.Graph().as_default() as tf_graph:
        tf.import_graph_def(frozen_graph, name='')
    return tf2onnx.tfonnx.process_tf_graph(
        tf_graph,
        opset=11,
        input_names=input_names,
        output_names=output_names
    )

In [5]:
test_model = create_AND_model()
print(test_model)

In [6]:
test_onnx = tf_to_onnx(test_model)
print(test_onnx)

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


In [9]:
# Copied from marabou helloworld
import sys
import numpy as np
import tensorflow
    
sys.path.append('/home/pidge/Masters/Marabou')
from maraboupy import Marabou, MarabouNetworkONNX

In [10]:
# restrict printing
options = Marabou.createOptions(verbosity = 0)

In [11]:
network = Marabou.read_onnx('/home/pidge/Masters/dissertation/marabou-testing/networks/and-gate/model.onnx')

In [13]:
inputVars = network.inputVars[0][0]
outputVars = network.outputVars[0]


In [14]:
network.setLowerBound(inputVars[0],-10.0)
network.setUpperBound(inputVars[0], 10.0)
network.setLowerBound(inputVars[1],-10.0)
network.setUpperBound(inputVars[1], 10.0)


In [17]:
network.setLowerBound(outputVars[1], -210.0)
network.setUpperBound(outputVars[1], 210.0)


sat
input 0 = -10.0
input 1 = -10.0
output 0 = 155.0
output 1 = 155.0
